In [326]:
# import packages
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import time
import geopy
import re
import csv
from geopy.distance import geodesic
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

# This section extracts postcodes from database and gets geolocation data from OneMap API

In [327]:
# load Michelin restaurants list
df_postcodes = pd.read_csv('restaurant_postcodes.csv', dtype = {'postcode': str})
df_postcodes = df_postcodes.drop('Unnamed: 0', axis=1)
#df_postcodes.columns = ['restaurant', 'postcode']
df_postcodes.head(10)

,restaurant,postcode,michelin_grade
0,Poh Cheu,150127,The MICHELIN Plate: Good cooking
1,New Rong Liang Ge Cantonese Roast Duck Double ...,182269,The MICHELIN Plate: Good cooking
2,Chef Kang's Noodle House,319579,"Bib Gourmand: good quality, good value cooking"
3,Guan Kee Fried Kway Teow,270020,"Bib Gourmand: good quality, good value cooking"
4,Shi Le Yuan,150085,The MICHELIN Plate: Good cooking
5,San Bao Soya Sauce Chicken,150085,The MICHELIN Plate: Good cooking
6,Hua Kee Chicken Rice,150085,The MICHELIN Plate: Good cooking
7,Fu Ming Cooked Food,150085,"Bib Gourmand: good quality, good value cooking"
8,Koka Wanton Noodles,198783,The MICHELIN Plate: Good cooking
9,R&B Express,229495,The MICHELIN Plate: Good cooking


In [328]:
# add Bishan J8 homebase
df_postcodes = df_postcodes.append({'restaurant': 'homebase_bishan_J8', 'postcode': '579837', 'michelin_grade':'nil'}, ignore_index=True)

In [329]:
df_postcodes.shape

(214, 3)

In [330]:
# load postcodes and coordinates

df_coord = pd.read_csv('Michelin_geolocations.csv', dtype = {'Unnamed: 0': str})
df_coord.head()

,Unnamed: 0,X,Y,address,found,lat,long,postal
0,188613,30398.327704,30976.306537,36 PURVIS STREET TALIB CENTRE SINGAPORE 188613,1,1.296413,103.854869,188613
1,089855,28779.228028,29100.721959,41 BUKIT PASOH ROAD SINGAPORE 089855,1,1.279451,103.840320,89855
2,238865,27990.885800,31907.302982,320 ORCHARD ROAD TANG PLAZA SINGAPORE 238865,8,1.304833,103.833237,238865
3,038982,30833.050591,30656.960845,2 TEMASEK BOULEVARD CONRAD CENTENNIAL SINGAPOR...,1,1.293525,103.858775,38982
4,389531,34069.288579,32921.889913,592 GEYLANG ROAD GEYLANG CONSERVATION AREA SIN...,1,1.314008,103.887854,389531


In [331]:
df = pd.merge(df_postcodes, df_coord, how="left", left_on='postcode', right_on='Unnamed: 0')
df.head()

,restaurant,postcode,michelin_grade,Unnamed: 0,X,Y,address,found,lat,long,postal
0,Poh Cheu,150127,The MICHELIN Plate: Good cooking,150127,24669.952213,29750.644187,127 BUKIT MERAH LANE 1 SINGAPORE 150127,1,1.285329,103.803397,150127
1,New Rong Liang Ge Cantonese Roast Duck Double ...,182269,The MICHELIN Plate: Good cooking,182269,30242.267039,31412.365483,269B QUEEN STREET CHENG YAN COURT SINGAPORE 18...,1,1.300357,103.853466,182269
2,Chef Kang's Noodle House,319579,"Bib Gourmand: good quality, good value cooking",319579,29689.494809,35500.335815,11 LORONG 3 TOA PAYOH JACKSON SQUARE SINGAPORE...,2,1.337327,103.848500,319579
3,Guan Kee Fried Kway Teow,270020,"Bib Gourmand: good quality, good value cooking",270020,22983.336348,32579.350652,20 GHIM MOH ROAD MARKET & HAWKER CENTRE (BLK 2...,1,1.310910,103.788241,270020
4,Shi Le Yuan,150085,The MICHELIN Plate: Good cooking,150085,26332.895124,29972.027731,85 REDHILL LANE REDHILL FOOD CENTRE SINGAPORE ...,1,1.287331,103.818339,150085


In [332]:
df = df.drop(columns=['Unnamed: 0', 'postal'])

In [333]:
df.head(20)

,restaurant,postcode,michelin_grade,X,Y,address,found,lat,long
0,Poh Cheu,150127,The MICHELIN Plate: Good cooking,24669.952213,29750.644187,127 BUKIT MERAH LANE 1 SINGAPORE 150127,1,1.285329,103.803397
1,New Rong Liang Ge Cantonese Roast Duck Double ...,182269,The MICHELIN Plate: Good cooking,30242.267039,31412.365483,269B QUEEN STREET CHENG YAN COURT SINGAPORE 18...,1,1.300357,103.853466
2,Chef Kang's Noodle House,319579,"Bib Gourmand: good quality, good value cooking",29689.494809,35500.335815,11 LORONG 3 TOA PAYOH JACKSON SQUARE SINGAPORE...,2,1.337327,103.848500
3,Guan Kee Fried Kway Teow,270020,"Bib Gourmand: good quality, good value cooking",22983.336348,32579.350652,20 GHIM MOH ROAD MARKET & HAWKER CENTRE (BLK 2...,1,1.310910,103.788241
4,Shi Le Yuan,150085,The MICHELIN Plate: Good cooking,26332.895124,29972.027731,85 REDHILL LANE REDHILL FOOD CENTRE SINGAPORE ...,1,1.287331,103.818339
5,San Bao Soya Sauce Chicken,150085,The MICHELIN Plate: Good cooking,26332.895124,29972.027731,85 REDHILL LANE REDHILL FOOD CENTRE SINGAPORE ...,1,1.287331,103.818339
6,Hua Kee Chicken Rice,150085,The MICHELIN Plate: Good cooking,26332.895124,29972.027731,85 REDHILL LANE REDHILL FOOD CENTRE SINGAPORE ...,1,1.287331,103.818339
7,Fu Ming Cooked Food,150085,"Bib Gourmand: good quality, good value cooking",26332.895124,29972.027731,85 REDHILL LANE REDHILL FOOD CENTRE SINGAPORE ...,1,1.287331,103.818339
8,Koka Wanton Noodles,198783,The MICHELIN Plate: Good cooking,31409.487724,32008.405282,861 NORTH BRIDGE ROAD NORTH BRIDGE ROAD MARKET...,2,1.305747,103.863954
9,R&B Express,229495,The MICHELIN Plate: Good cooking,28678.194951,32720.364830,500 CLEMENCEAU AVENUE NORTH NEWTON FOOD CENTRE...,3,1.312186,103.839412


In [334]:
#retain only the 1 Michelin starred places and origin
df =df.loc[(df['michelin_grade'] == 'nil') | (df['michelin_grade'] == 'Bib Gourmand: good quality, good value cooking')]
#df =df.loc[(df['michelin_grade'] == 'One MICHELIN Star: High quality cooking, worth a stop!') | (df['michelin_grade'] == 'nil') | (df['michelin_grade'] == 'Two MICHELIN Stars: Excellent cooking, worth a detour!') | (df['michelin_grade'] == 'Bib Gourmand: good quality, good value cooking')]

In [335]:
df.shape

(57, 9)

# This section finds the best paths

In [336]:
### CALCULATE DISTANCE MATRIX

# create list of postcodes
unique_postcodes = df['postcode'].unique()
print(unique_postcodes)
origin_dest_distance = {}

# for each address
for i in unique_postcodes:
    origin_dest = {}
    origin_row = df.loc[df['postcode']==i]
    origin_lat = origin_row.iloc[0]['lat']
    origin_lon = origin_row.iloc[0]['long']
    for j in unique_postcodes:
        dest_row = df.loc[df['postcode']==j]
        dest_lat = dest_row.iloc[0]['lat']
        dest_lon = dest_row.iloc[0]['long']
        # find distance between origin and destination
        dist = geodesic((origin_lat, origin_lon), (dest_lat, dest_lon)).meters
        origin_dest[j] = dist
    origin_dest_distance[i] = origin_dest

origin_dest_distance_df = pd.DataFrame(origin_dest_distance).transpose()

origin_dest_distance_df = origin_dest_distance_df.sort_index(ascending=True, axis=1)
origin_dest_distance_df = origin_dest_distance_df.sort_index(ascending=True, axis=0)

['319579' '270020' '150085' '180270' '068815' '270040' '389359' '247792'
 '320091' '168898' '151115' '229495' '390051' '050335' '460208' '150120'
 '150006' '389531' '218591' '270044' '088518' '089109' '058972' '088539'
 '068803' '600253' '310127' '140159' '069184' '051531' '069111' '389589'
 '209379' '428820' '188613' '079331' '179937' '237995' '059386' '207466'
 '089137' '169850' '579837']


In [337]:
origin_dest_distance_df.shape

(43, 43)

In [338]:
vrp_distance_matrix = origin_dest_distance_df.values.tolist()

In [339]:
# find Bishan Junction 8 home index
home_postcode = '579837'
home_index_vrp = list(origin_dest_distance_df.columns).index(home_postcode)
print(home_index_vrp)

41


## This section sets the capacity constraints and number of restaurants per postcode required (if there is more than one)

In [340]:
postal_count_vrp4 = pd.DataFrame(df.groupby('postcode')['postcode'].count())

In [341]:
postal_count_vrp4 = postal_count_vrp4.rename(columns={"postcode": "count"})

In [342]:
# this shows the number of restaurants at each postcode
# Some postcodes have 2 or more restaurants
postal_count_vrp4 = postal_count_vrp4.sort_values(by='count', ascending = False)
postal_count_vrp4.head(15)

,count
postcode,
069111,4
051531,3
168898,3
229495,2
150006,2
390051,2
270020,2
151115,2
320091,2


In [343]:
# save the postcodes with multiple food establishments, to include in the VRP model later
postcodes_multiple_outlets_vrp4 = postal_count_vrp4.loc[(postal_count_vrp4['count'] >1)]

In [344]:
postcodes_multiple_outlets_dict_vrp4 = postcodes_multiple_outlets_vrp4.to_dict("index")

In [345]:
postcodes_multiple_outlets_dict_vrp4

{'069111': {'count': 4},
 '051531': {'count': 3},
 '168898': {'count': 3},
 '229495': {'count': 2},
 '150006': {'count': 2},
 '390051': {'count': 2},
 '270020': {'count': 2},
 '151115': {'count': 2},
 '320091': {'count': 2},
 '150120': {'count': 2}}

In [346]:
# distance matrix
origin_dest_distance_df

,050335,051531,058972,059386,068803,068815,069111,069184,079331,088518,...,319579,320091,389359,389531,389589,390051,428820,460208,579837,600253
050335,0.000000,448.593682,107.875986,885.396278,882.823599,810.717736,506.876568,272.676939,573.736413,537.406003,...,6120.481859,4703.511380,5247.780636,6088.077978,6170.926028,5550.823123,7372.422298,10899.014423,7536.657139,13624.286820
051531,448.593682,0.000000,394.615312,442.935126,846.818831,925.261575,677.539020,562.247179,899.570729,901.130321,...,5761.110554,4304.520474,4805.016085,5653.272025,5735.835697,5125.390018,6979.014804,10486.888150,7181.112065,13719.931918
058972,107.875986,394.615312,0.000000,815.837453,947.921449,899.845208,597.448975,372.246837,681.452880,644.836629,...,6018.168214,4608.437099,5199.105148,6047.652237,6130.251412,5517.630968,7357.582444,10874.518824,7433.722842,13551.757059
059386,885.396278,442.935126,815.837453,0.000000,1103.750945,1266.307513,1071.001487,996.563831,1326.285791,1336.545155,...,5361.655001,3878.561207,4401.078694,5267.228094,5349.094918,4758.763514,6662.152832,10137.567083,6782.963392,13725.370308
068803,882.823599,846.818831,947.921449,1103.750945,0.000000,291.860958,439.212605,657.660845,698.619321,782.926287,...,6420.944792,4882.730981,4891.739739,5653.906282,5737.939984,5066.447910,6750.190552,10326.441149,7841.607512,14498.955233
068815,810.717736,925.261575,899.845208,1266.307513,291.860958,0.000000,303.859901,543.913204,453.990376,547.182212,...,6622.098966,5107.995215,5182.717926,5945.612495,6029.651969,5357.050152,7027.587268,10608.786336,8043.540105,14420.418429
069111,506.876568,677.539020,597.448975,1071.001487,439.212605,303.859901,0.000000,242.032869,310.741411,373.507931,...,6428.781391,4947.002332,5195.051503,5989.187744,6072.946193,5418.886860,7150.437453,10713.793883,7849.655622,14121.342720
069184,272.676939,562.247179,372.246837,996.563831,657.660845,543.913204,242.032869,0.000000,340.686138,340.003934,...,6316.319117,4866.767536,5258.149697,6074.310719,6157.723052,5518.508143,7291.246569,10839.742771,7735.309151,13879.317921
079331,573.736413,899.570729,681.452880,1326.285791,698.619321,453.990376,310.741411,340.686138,0.000000,95.321316,...,6656.888286,5202.379364,5505.725466,6299.036359,6382.823336,5726.703729,7443.667158,11013.190496,8075.958885,14041.389943
088518,537.406003,901.130321,644.836629,1336.545155,782.926287,547.182212,373.507931,340.003934,95.321316,0.000000,...,6645.956177,5205.110955,5559.182546,6359.580416,6443.276266,5791.544087,7520.787603,11086.564333,8063.846100,13952.182522


In [347]:
# create weights for each outlet at each postcode
weight_list_vrp4 = []

for x in list(origin_dest_distance_df.columns):
    if x not in postcodes_multiple_outlets_dict_vrp4.keys():
        if x not in [home_postcode]:
            weight_list_vrp4.append(1)
            
            print("weight_list:", weight_list_vrp4)
        elif x in [home_postcode]:
            weight_list_vrp4.append(0)
            
            
    else:
        weight_list_vrp4.append(postcodes_multiple_outlets_dict_vrp4[x]['count'])
        
        print("2 outlets in this postcode:", weight_list_vrp4)

weight_list: [1]
2 outlets in this postcode: [1, 3]
weight_list: [1, 3, 1]
weight_list: [1, 3, 1, 1]
weight_list: [1, 3, 1, 1, 1]
weight_list: [1, 3, 1, 1, 1, 1]
2 outlets in this postcode: [1, 3, 1, 1, 1, 1, 4]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1]
2 outlets in this postcode: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1]
2 outlets in this postcode: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2]
2 outlets in this postcode: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2]
2 outlets in this postcode: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 3]
weight_list: [1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 3, 1]


In [348]:
len(weight_list_vrp4)

43

In [349]:
print(weight_list_vrp4)

[1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1]


### This codes solves the VRP with capacity constraints

In [350]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = origin_dest_distance_df.values.tolist()
    data['demands'] = weight_list_vrp4
    data['vehicle_capacities'] = [11, 11, 11, 11, 12]
    data['num_vehicles'] = 5
    data['depot'] = home_index_vrp
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    # hack a data array
    saving_solution = {}
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes
    
def main_vrp4_cap():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        x = get_routes(solution, routing, manager)
        print_solution(data, manager, routing, solution)
        return(x)
    
#if __name__ == '__main__':
#    main()

In [351]:
x_vrp4_cap = main_vrp4_cap()

Route for vehicle 0:
 41 Load(0) ->  15 Load(1) ->  14 Load(3) ->  16 Load(5) ->  13 Load(6) ->  30 Load(7) ->  31 Load(8) ->  29 Load(10) ->  42 Load(11) ->  41 Load(11)
Distance of the route: 31840m
Load of the route: 11

Route for vehicle 1:
 41 Load(0) ->  33 Load(1) ->  27 Load(2) ->  3 Load(3) ->  20 Load(4) ->  22 Load(5) ->  21 Load(6) ->  24 Load(7) ->  23 Load(8) ->  25 Load(9) ->  34 Load(11) ->  41 Load(11)
Distance of the route: 14699m
Load of the route: 11

Route for vehicle 2:
 41 Load(0) ->  40 Load(1) ->  39 Load(2) ->  37 Load(3) ->  36 Load(4) ->  38 Load(6) ->  35 Load(7) ->  4 Load(8) ->  5 Load(9) ->  0 Load(10) ->  2 Load(11) ->  41 Load(11)
Distance of the route: 30437m
Load of the route: 11

Route for vehicle 3:
 41 Load(0) ->  1 Load(3) ->  7 Load(4) ->  9 Load(5) ->  10 Load(6) ->  8 Load(7) ->  6 Load(11) ->  41 Load(11)
Distance of the route: 16801m
Load of the route: 11

Route for vehicle 4:
 41 Load(0) ->  12 Load(1) ->  11 Load(2) ->  19 Load(3) ->  18 L

In [352]:
route_postcode_saver_vrp4 = {}
for i in range(len(x_vrp4_cap)):
    route = []
    for j in x_vrp4_cap[i]:
        #print(j)
        route.append(origin_dest_distance_df.columns[j])
        #print(route)
    route_postcode_saver_vrp4[i] = route
print (route_postcode_saver_vrp4)

{0: ['579837', '150085', '150006', '150120', '140159', '270040', '270044', '270020', '600253', '579837'], 1: ['579837', '319579', '237995', '059386', '179937', '188613', '180270', '209379', '207466', '218591', '320091', '579837'], 2: ['579837', '460208', '428820', '389589', '389531', '390051', '389359', '068803', '068815', '050335', '058972', '579837'], 3: ['579837', '051531', '069184', '088518', '088539', '079331', '069111', '579837'], 4: ['579837', '089137', '089109', '169850', '168898', '151115', '247792', '229495', '310127', '579837']}


### Export VRP 4 routes

In [353]:
with open('michelin_routes.csv', 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for i in range(len(route_postcode_saver_vrp4)):
        wr.writerow(route_postcode_saver_vrp4[i])